# Spark: Getting Started
 * These instructions require a Mac with [Anaconda3](https://anaconda.com/) and [Homebrew](https://brew.sh/) installed.
 * Useful for small data only. For larger data, try [Databricks](https://databricks.com/).

## Step 0: Prerequisites & Installation

Run these commands in your terminal (just once).

```bash
# Make Homebrew aware of old versions of casks
brew tap caskroom/versions

# Install Java 1.8 (OpenJDK 8)
brew cask install adoptopenjdk8

# Install the current version of Spark
brew install apache-spark

# Install Py4J (connects PySpark to the Java Virtual Machine)
pip install py4j

# Add JAVA_HOME to .bash_profile (makes Java 1.8 your default JVM)
echo "
export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)" >> ~/.bash_profile

# Add SPARK_HOME to .bash_profile
export SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.3/libexec
echo "
export SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.3/libexec" >> ~/.bash_profile

# Add PySpark to PYTHONPATH
echo "
export PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH" >> ~/.bash_profile

# Update current environment
source ~/.bash_profile

```

## Step 1: Create a SparkSession with a SparkContext

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
spark

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Step 2: Download some Amazon reviews (Toys & Games)

In [7]:
# Download data (run this only once)
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
#!gunzip reviews_Toys_and_Games_5.json.gz

## Step 3: Create a Spark DataFrame

In [23]:
df = spark.read.json('reviews_Toys_and_Games_5.json')

In [24]:
df.persist()

DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]

In [52]:
df.limit(5).toPandas()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600
1,0439893577,"[1, 1]",4.0,Love the magnet easel... great for moving to d...,"03 28, 2014",A8R62G708TSCM,Candace,it works pretty good for moving to different a...,1395964800
2,0439893577,"[1, 1]",5.0,Both sides are magnetic. A real plus when you...,"01 28, 2013",A21KH420DK0ICA,capemaychristy,love this!,1359331200
3,0439893577,"[0, 0]",5.0,Bought one a few years ago for my daughter and...,"02 8, 2014",AR29QK6HPFYZ4,dcrm,Daughters love it,1391817600
4,0439893577,"[1, 1]",4.0,I have a stainless steel refrigerator therefor...,"05 5, 2014",ACCH8EOML6FN5,DoyZ,Great to have so he can play with his alphabet...,1399248000


In [21]:
df.count()

167597

In [25]:
reviews_df = df[['asin', 'overall']]

In [26]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [27]:
show(reviews_df)

,asin,overall
0,0439893577,5.0
1,0439893577,4.0
2,0439893577,5.0
3,0439893577,5.0
4,0439893577,4.0


In [28]:
reviews_df.count()

167597

In [29]:
show(reviews_df)

,asin,overall
0,0439893577,5.0
1,0439893577,4.0
2,0439893577,5.0
3,0439893577,5.0
4,0439893577,4.0


In [31]:
sorted_review_df = reviews_df.sort('overall')

In [32]:
show(sorted_review_df)

,asin,overall
0,0786955708,1.0
1,0976990709,1.0
2,0963679600,1.0
3,0786955708,1.0
4,0974665207,1.0


In [33]:
import pyspark.sql.functions as F

In [35]:
counts = reviews_df.agg(F.countDistinct('overall'))

In [44]:
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""

In [45]:
reviews_df.createOrReplaceTempView('reviews')

In [46]:
output = spark.sql(query)

In [47]:
show(output, n=1000)

,overall,count(1)
0,1.0,4707
1,2.0,6298
2,3.0,16357
3,4.0,37445
4,5.0,102790


In [48]:
reviews_df.rdd

MapPartitionsRDD[141] at javaToPython at NativeMethodAccessorImpl.java:0

### Count the words in the first row

In [53]:
row_one = df.first()

In [56]:
row_one

Row(asin='0439893577', helpful=[0, 0], overall=5.0, reviewText='I like the item pricing. My granddaughter wanted to mark on it but I wanted it just for the letters.', reviewTime='01 29, 2014', reviewerID='A1VXOAVRGKGEAK', reviewerName='Angie', summary='Magnetic board', unixReviewTime=1390953600)

In [61]:
def word_count(text):
    return len(text.split())

In [62]:
word_count(row_one['reviewText'])

20

In [67]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

In [63]:
review_text_col = df['reviewText']

In [69]:
counts_df = df.withColumn('wordCount', word_count_udf(review_text_col))

In [70]:
show(counts_df).T

,0,1,2,3,4
asin,0439893577,0439893577,0439893577,0439893577,0439893577
helpful,"[0, 0]","[1, 1]","[1, 1]","[0, 0]","[1, 1]"
overall,5,4,5,5,4
reviewText,I like the item pricing. My granddaughter want...,Love the magnet easel... great for moving to d...,Both sides are magnetic. A real plus when you...,Bought one a few years ago for my daughter and...,I have a stainless steel refrigerator therefor...
reviewTime,"01 29, 2014","03 28, 2014","01 28, 2013","02 8, 2014","05 5, 2014"
reviewerID,A1VXOAVRGKGEAK,A8R62G708TSCM,A21KH420DK0ICA,AR29QK6HPFYZ4,ACCH8EOML6FN5
reviewerName,Angie,Candace,capemaychristy,dcrm,DoyZ
summary,Magnetic board,it works pretty good for moving to different a...,love this!,Daughters love it,Great to have so he can play with his alphabet...
unixReviewTime,1390953600,1395964800,1359331200,1391817600,1399248000
wordCount,20,22,76,31,47


In [67]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

df.createOrReplaceTempView('reviews')
spark.udf.register('word_count', word_count_udf)

In [84]:
query = """
SELECT asin, overall, reviewText, word_count(reviewText) AS wordCount
FROM reviews
"""

In [85]:
counts_df = spark.sql(query)

In [86]:
show(counts_df)

,asin,overall,reviewText,wordCount
0,0439893577,5.0,I like the item pricing. My granddaughter want...,20
1,0439893577,4.0,Love the magnet easel... great for moving to d...,22
2,0439893577,5.0,Both sides are magnetic. A real plus when you...,76
3,0439893577,5.0,Bought one a few years ago for my daughter and...,31
4,0439893577,4.0,I have a stainless steel refrigerator therefor...,47


In [88]:
def count_all_the_things(text):
    return [len(text), len(text.split())]

In [89]:
from pyspark.sql.types import ArrayType, IntegerType
count_udf = F.udf(count_all_the_things, ArrayType(IntegerType()))

In [90]:
counts_df = df.withColumn('counts', count_udf(df['reviewText']))

In [99]:
show(counts_df, 1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [98]:
slim_counts_df = (
    df.drop('reviewTime')
      .drop('helpful')
      .withColumn('counts', count_udf(df['reviewText']))
      .drop('reviewText')
)

In [101]:
show(slim_counts_df, n=1)

,asin,overall,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,5.0,A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [96]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
